In [1]:
import pandas as pd
import numpy as np
import mysql.connector as connection
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# commnet table
# news table (commnet table 추출 결과 보고 범위 설정해서 가져옴)

'''
'database ='
'query ='에서 db이름 변경해주세요.
'''

try:
    database = connection.connect(host="localhost", database = 'newsdb', user="root", passwd="1111", use_pure=True)
    query1 = "SELECT * FROM newsdb.tb_comment WHERE ID > 500000 AND ID < 600000;" # 댓글 데이터 10만개 사용
    query2 = "SELECT * FROM newsdb.tb_news where ID >= 498440 AND ID <= 572714;" # 댓글 데이터에서 NewsID 확인 후 해당 news 불러옴
    user = pd.read_sql(query1,database)
    news = pd.read_sql(query2,database)
    user['Content'] = user['Content'].str.strip() # 공백 처리 덜 된 부분 strip
    news['Content'] = news['Content'].str.strip()
    database.close()
    
except Exception as e:
    database.close()
    print(str(e))

In [3]:
user.head(1)

,ID,NewsID,UserID,WritedAt,Content
0,500001,498440,813055,2022-11-04 00:57:21,"속사정을 잘 아셔야합니다. 현재 해피파트너즈에게 민노총소속과 한노총소속,무소속이 있..."


In [4]:
news.head(1)

,ID,DomainID,MainCategory,SubCategory,WritedAt,Title,Content,URL,PhotoURL,Writer,Stickers,Press
0,498440,1,경제,경제일반,2022-11-03 17:18:00,'노조 탈퇴 강요' SPC PB파트너즈 법인·임직원 28명 검찰 송치,민주노총 소속 파리바게뜨 제빵기사를 대상으로 노동조합 탈퇴를 강요하는 등 부당노동행...,https://n.news.naver.com/mnews/article/214/000...,https://imgnews.pstatic.net/image/214/2022/11/...,임상재(limsj@mbc.co,"{'놀랍네요': 1, '추천해요': 16, '유익해요': 2, '분석이탁월해요': ...",MBC


In [5]:
# news dataframe의 ID 컬럼을 NewsID로 변경 (NewsID 기준으로 user dataframe과 병합하기 위해 변경)
news = news.rename(columns={'ID' : 'NewsID'})

In [6]:
news.head(1)

,NewsID,DomainID,MainCategory,SubCategory,WritedAt,Title,Content,URL,PhotoURL,Writer,Stickers,Press
0,498440,1,경제,경제일반,2022-11-03 17:18:00,'노조 탈퇴 강요' SPC PB파트너즈 법인·임직원 28명 검찰 송치,민주노총 소속 파리바게뜨 제빵기사를 대상으로 노동조합 탈퇴를 강요하는 등 부당노동행...,https://n.news.naver.com/mnews/article/214/000...,https://imgnews.pstatic.net/image/214/2022/11/...,임상재(limsj@mbc.co,"{'놀랍네요': 1, '추천해요': 16, '유익해요': 2, '분석이탁월해요': ...",MBC


In [7]:
# 안 쓸 컬럼 삭제
news.drop(['PhotoURL', 'Writer', 'Stickers', 'Press'], axis=1, inplace=True)
news.head(1)

,NewsID,DomainID,MainCategory,SubCategory,WritedAt,Title,Content,URL
0,498440,1,경제,경제일반,2022-11-03 17:18:00,'노조 탈퇴 강요' SPC PB파트너즈 법인·임직원 28명 검찰 송치,민주노총 소속 파리바게뜨 제빵기사를 대상으로 노동조합 탈퇴를 강요하는 등 부당노동행...,https://n.news.naver.com/mnews/article/214/000...


In [8]:
user.shape

(99999, 5)

In [9]:
# NewsID 기준으로 user dataframe과 news dataframe 병합
user_news = pd.merge(user, news, on='NewsID')

In [10]:
user_news.shape

(50665, 12)

In [11]:
user_news.head(1)

,ID,NewsID,UserID,WritedAt_x,Content_x,DomainID,MainCategory,SubCategory,WritedAt_y,Title,Content_y,URL
0,500001,498440,813055,2022-11-04 00:57:21,"속사정을 잘 아셔야합니다. 현재 해피파트너즈에게 민노총소속과 한노총소속,무소속이 있...",1,경제,경제일반,2022-11-03 17:18:00,'노조 탈퇴 강요' SPC PB파트너즈 법인·임직원 28명 검찰 송치,민주노총 소속 파리바게뜨 제빵기사를 대상으로 노동조합 탈퇴를 강요하는 등 부당노동행...,https://n.news.naver.com/mnews/article/214/000...


In [12]:
# column 이름 변경
user_news = user_news.rename(columns={'WritedAt_x' : 'WritedAt_Commnet', 'Content_x' : 'Content_Commnet', 'WritedAt_y' : 'WritedAt_News', 'Content_y' : 'Content_News'})

In [13]:
user_news.head(1)

,ID,NewsID,UserID,WritedAt_Commnet,Content_Commnet,DomainID,MainCategory,SubCategory,WritedAt_News,Title,Content_News,URL
0,500001,498440,813055,2022-11-04 00:57:21,"속사정을 잘 아셔야합니다. 현재 해피파트너즈에게 민노총소속과 한노총소속,무소속이 있...",1,경제,경제일반,2022-11-03 17:18:00,'노조 탈퇴 강요' SPC PB파트너즈 법인·임직원 28명 검찰 송치,민주노총 소속 파리바게뜨 제빵기사를 대상으로 노동조합 탈퇴를 강요하는 등 부당노동행...,https://n.news.naver.com/mnews/article/214/000...


In [14]:
user_news.columns

Index(['ID', 'NewsID', 'UserID', 'WritedAt_Commnet', 'Content_Commnet',
       'DomainID', 'MainCategory', 'SubCategory', 'WritedAt_News', 'Title',
       'Content_News', 'URL'],
      dtype='object')

In [15]:
# 댓글작성자 - 해당뉴스에 대한 pivot table 생성(index는 NewsID, Column은 UserID로 받는 데이터프레임)
news_user_rating = user_news.pivot_table('ID', index = 'NewsID', columns='UserID')

In [16]:
# NaN값(유저가 기사에 댓글을 작성하지 않은 경우)은 0으로 채움
news_user_rating.fillna(0, inplace=True)
news_user_rating.head(5)

UserID,367572,367575,367576,367578,367583,367584,367585,367586,367588,367589,...,976611,976612,976619,976620,976627,976631,976634,976635,976644,976650
NewsID,,,,,,,,,,,,,,,,,,,,,
498440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
news_user_rating.shape

(15095, 29817)

In [18]:
# cosine 유사도 구하기
item_based_collaboration = cosine_similarity(news_user_rating)
item_based_collaboration

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [19]:
item_based_collaboration = pd.DataFrame(data = item_based_collaboration, index = news_user_rating.index, columns = news_user_rating.index)

In [20]:
item_based_collaboration.head(1)

NewsID,498440,498444,498446,498461,498465,498469,498470,498472,498475,498478,...,572478,572479,572480,572486,572487,572511,572514,572712,572713,572714
NewsID,,,,,,,,,,,,,,,,,,,,,
498440,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.223606,0.0,0.0,0.0,0.0


In [21]:
# item 간 유사도 측정 후 get_item_based_collaboration 함수를 통해 비슷한 news 추천

def get_item_based_collaboration(NewsID):
    return item_based_collaboration[NewsID].sort_values(ascending=False).iloc[0:15]

In [22]:
# 기존 news 데이터프레임과 병합하여, 제목과 URL 및 카테고리까지 표시할 수 있게 함
results = get_item_based_collaboration(526503) # NewsID

results_df = pd.DataFrame({'NewsID':results.index, 'Value':results.values})
results_df = results_df[results_df['Value'] > 0] # Value값이 0보다 작은 경우는 출력 X

final_df = pd.merge(results_df, news, on='NewsID')
final_df[['NewsID', 'Value', 'Title', 'URL', 'MainCategory', 'SubCategory']]

,NewsID,Value,Title,URL,MainCategory,SubCategory
0,526503,1.000000,데드라인 임박 '공시가 현실화 수정안'…이달말 ‘세부담 완화카드’ 나올까,https://n.news.naver.com/mnews/article/421/000...,경제,부동산
1,540078,0.577353,자금시장 불안에 한 달 만에 또 대책…은행 대출여력 8.5조 확충,https://n.news.naver.com/mnews/article/015/000...,경제,금융
2,540526,0.577353,"홍경식 한은 통화정책국장 ""당분간 금리인상 기조 이어나갈 필요""",https://n.news.naver.com/mnews/article/119/000...,경제,금융
3,533444,0.577353,[Why&Next]D-3 금통위 키워드 셋…성장률·최종금리·소수의견,https://n.news.naver.com/mnews/article/277/000...,경제,금융
4,523085,0.408252,"건정연 ""내년 건설투자 줄고, 수도권 매매·전세가격 3~4% 하락""",https://n.news.naver.com/mnews/article/277/000...,경제,부동산
5,526789,0.408251,아파트 매매 5건 중 거의 1건꼴로 '직거래'…역대 최고치,https://n.news.naver.com/mnews/article/001/001...,경제,부동산
6,571217,0.408250,"건설정책연구원 ""내년 수도권 아파트값 3∼4% 하락…내후년 저점""",https://n.news.naver.com/mnews/article/374/000...,경제,경제일반
7,524667,0.408250,10월 미분양 한달 새 13.5% 증가...올해 주택 매매량은 전년 대비 반토막,https://n.news.naver.com/mnews/article/023/000...,경제,부동산
8,499711,0.408249,집값 폭락에…공시가격 현실화율 목표 조정 1년 미룰 듯,https://n.news.naver.com/mnews/article/448/000...,경제,경제일반
9,554277,0.408249,"中, '기준금리격' LPR 또 동결…자본유출 우려(상보)",https://n.news.naver.com/mnews/article/277/000...,경제,글로벌경제


In [23]:
# 아이템 기반 협업 필터링 구현 과정

# 데이터베이스에서 데이터 불러오기
# commnet table(=user 데이터프레임으로 사용) / news table(=news 데이터프레임으로 사용)
# news table의 경우 commnet table 추출 결과에서 NewsID 값 보고 범위 설정해서 가져옴
# news 데이터프레임의 ID 컬럼을 NewsID로 변경 (user 데이터프레임과 병합하기 위해)
# 공통으로 가지고 있는 NewsID를 기준으로 병합
# 댓글작성자 - 해당뉴스에 대한 pivot table 생성(index는 NewsID, Column은 UserID로 받는 데이터프레임)
# NaN값은 0으로 채우기
# 유저가 해당 기사에 댓글을 단 경우 1, 아닌 경우 0으로 반환
# cosine 유사도 구하기
# item 간 유사도 측정 후 get_item_based_collaboration 을 통해 비슷한 news 추천
# 마지막 과정에서 기존 news 데이터프레임과 병합하여, 제목과 URL 및 카테고리까지 표시할 수 있게 함

##### 아이템 기반 협업 필터링 구현 과정

* DB에서 데이터 불러오기

* commnet table(=user 데이터프레임으로 사용) / news table(=news 데이터프레임으로 사용)
* DB 데이터 확인 시 comment table의 NewsID와 news table의 ID 값이 동일함을 확인
* news table의 경우 commnet table 추출 결과에서 NewsID 값 확인 후 범위 설정해서 가져옴

* news 데이터프레임의 ID 컬럼을 NewsID로 변경 (user 데이터프레임과 병합하기 위해)
* 공통으로 가지고 있는 NewsID를 기준으로 병합
* 댓글작성자 - 해당뉴스에 대한 pivot table 생성(index는 NewsID, Column은 UserID로 받는 데이터프레임)
* NaN값(유저가 기사에 댓글을 작성하지 않은 경우)은 0으로 채움
* 유저가 해당 기사에 댓글을 단 경우 1, 아닌 경우 0으로 반환

* cosine 유사도 구하기
* item 간 유사도 측정 후 get_item_based_collaboration 함수를 통해 비슷한 news 추천
* 마지막 과정에서 기존 news 데이터프레임과 병합하여, 제목과 URL 및 카테고리까지 표시할 수 있게 함